In [1]:
import pandas as pd
import numpy as np

# Load Training Data

In [2]:
train_data = pd.read_csv("ModelData/obs_train.csv")
for c in train_data.columns:
    train_data.rename(columns={c: "C_" + c}, inplace=True)

In [3]:
test_data = pd.read_csv("ModelData/obs_test.csv")
for c in test_data.columns:
    test_data.rename(columns={c: "C_" + c}, inplace=True)

# Build Baseline Regression Model

In [4]:
# predict probabilities with a multinomial logistic regression model
from sklearn.linear_model import LogisticRegression

# define the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

# Train data
X = train_data[["C_stock", "C_1_month_return", "C_6_month_return", "C_12_month_return", "C_1_month_volatility", "C_6_month_volatility", "C_12_month_volatility"]]
y = train_data[["C_label_1m"]]

# fit the model on the whole dataset
model.fit(X, y)

C:\Users\yvesd\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(max_iter=1000, multi_class='multinomial')

# Test Baseline Logit Model

In [5]:
# predict a multinomial probability distribution
yhat = model.predict_proba(test_data[["C_stock", "C_1_month_return", "C_6_month_return", "C_12_month_return", "C_1_month_volatility", "C_6_month_volatility", "C_12_month_volatility"]])

In [6]:
actual_vs_pred = pd.concat(
    [test_data["C_label_1m"], pd.DataFrame(yhat)],
    axis=1
)

In [7]:
actual_vs_pred["pred"] = actual_vs_pred[[0, 1, 2]].apply(
    lambda x: 0 if np.max(x) == x[0] else 1 if np.max(x) == x[1] else 2,
    axis=1
)

In [8]:
actual_vs_pred["correct"] = actual_vs_pred[["C_label_1m", "pred"]].apply(lambda x: x[0] == x[1], axis=1)

Overall Accuracy

In [9]:
actual_vs_pred.correct.sum() / len(actual_vs_pred.index)

0.4369369369369369

Precision

In [10]:
print("Precision Negative Returns: {}".format(
    len(
        actual_vs_pred[(actual_vs_pred.pred == 0) & (actual_vs_pred.C_label_1m == 0)]
    ) / len(
        actual_vs_pred[(actual_vs_pred.pred == 0)].index
    )
))

Precision Negative Returns: 0.4365256124721604


In [11]:
print("Precision Neutral Returns: {}".format(
    len(
        actual_vs_pred[(actual_vs_pred.pred == 1) & (actual_vs_pred.C_label_1m == 1)]
    ) / len(
        actual_vs_pred[(actual_vs_pred.pred == 1)].index
    )
))

Precision Neutral Returns: 0.46229508196721314


In [12]:
print("Precision Positive Returns: {}".format(
    len(
        actual_vs_pred[(actual_vs_pred.pred == 2) & (actual_vs_pred.C_label_1m == 2)]
    ) / len(
        actual_vs_pred[(actual_vs_pred.pred == 2)].index
    )
))

Precision Positive Returns: 0.38095238095238093


Recall

In [13]:
print("Recall Negative Returns: {}".format(
    len(
        actual_vs_pred[(actual_vs_pred.pred == 0) & (actual_vs_pred.C_label_1m == 0)]
    ) / len(
        actual_vs_pred[(actual_vs_pred.C_label_1m == 0)].index
    )
))

Recall Negative Returns: 0.44144144144144143


In [14]:
print("Recall Neutral Returns: {}".format(
    len(
        actual_vs_pred[(actual_vs_pred.pred == 1) & (actual_vs_pred.C_label_1m == 1)]
    ) / len(
        actual_vs_pred[(actual_vs_pred.C_label_1m == 1)].index
    )
))

Recall Neutral Returns: 0.6351351351351351


In [15]:
print("Recall Positive Returns: {}".format(
    len(
        actual_vs_pred[(actual_vs_pred.pred == 2) & (actual_vs_pred.C_label_1m == 2)]
    ) / len(
        actual_vs_pred[(actual_vs_pred.C_label_1m == 2)].index
    )
))

Recall Positive Returns: 0.23423423423423423
